In [110]:
from pulp import *
import pandas as pd
from pulp import LpMinimize

In [2]:
pip install pulp


  Obtaining dependency information for pulp from https://files.pythonhosted.org/packages/09/d7/57e71e11108203039c895643368c0d1a99fe719a6a80184edf240c33d25f/PuLP-2.8.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB 279.3 kB/s eta 0:01:04
   ---------------------------------------- 0.1/17.7 MB 456.6 kB/s eta 0:00:39
   ---------------------------------------- 0.2/17.7 MB 908.0 kB/s eta 0:00:20
    --------------------------------------- 0.3/17.7 MB 1.3 MB/s eta 0:00:14
   - -------------------------------------- 0.5/17.7 MB 1.8 MB/s eta 0:00:10
   -- ------------------------------------- 1.0/17.7 MB 3.0 MB/s 

In [24]:
#데이터 불러오기
Bus_info_data = pd.read_csv("Bus_info.csv", encoding='cp949')

In [25]:
#데이터 보기
Bus_info_data

,버스번호,정류장번호,정류장,막차시간,안암역->차량이동시간(오후24:00 출발 기준),택시비,거리,도보시간
0,1,1,매헌시민의숲.양재꽃시장,23:52,26,21234.35115,15.2,359
1,1,2,교육개발원입구,23:53,30,19631.29771,13.7,373
2,1,3,양재역.서초문화예술회관,23:54,24,19417.55725,13.5,380
3,1,4,뱅뱅사거리(중),23:57,28,18776.33588,12.9,391
4,1,5,래미안아파트.파이넨셜뉴스,23:58,27,18241.98473,12.4,398
...,...,...,...,...,...,...,...,...
86,3,22,용인대,0:53,52,58532.06107,50.1,239
87,3,23,동원로알듀크,0:57,54,58638.93130,50.2,230
88,3,24,이마트.상공회의소,0:58,52,59814.50382,51.3,244
89,3,25,역북동행정복지센터,0:58,54,60455.72519,51.9,238


In [148]:
#시간단위를 분으로 바꿔주기
def time_to_minutes(time_str):
    hours, minutes = map(int, time_str.split(':'))
    if hours == 23: #24시 이전은 -로 표시
        return -60 + minutes
    return hours * 60 + minutes

## 데이터로부터 파라미터 설정

In [161]:
#데이터로부터 파라미터 설정

#모든 정류장 이름들
bus_stops = list(Bus_info_data['정류장']) 
#모든 버스 종류
buses = [h for h in Bus_info_data['버스번호'].unique()] #모든 버스 종류
#버스 별 정류장
stops_each_bus = {}
for h in H:
    stops_each_bus[h] = Bus_info_data[Bus_info_data['버스번호'] == h]['정류장']
#출발지 부터 해당 정류장까지 택시비
taxi_costs = dict(zip(Bus_info_data['정류장'], Bus_info_data['택시비'])) 
#출발지부터 해당 정류장까지 택시시간
taxi_times = dict(zip(Bus_info_data['정류장'], Bus_info_data['안암역->차량이동시간(오후24:00 출발 기준)']))
#i 정류장부터 j 정류장까지 버스 시간
bus_times = {}
for i in Bus_info_data['정류장']:
    for j in Bus_info_data['정류장']:
        last_time_of_i = time_to_minutes(Bus_info_data.loc[Bus_info_data['정류장'] == i, '막차시간'].values[0])
        last_time_of_j = time_to_minutes(Bus_info_data.loc[Bus_info_data['정류장'] == j, '막차시간'].values[0])
        bus_times[(i,j)] = last_time_of_j - last_time_of_i
#해당정류장부터 도착지까지 도보시간
walk_times = dict(zip(Bus_info_data['정류장'], Bus_info_data['도보시간'])) 
#해당정류장에서의 버스 막차시간
last_bus_times = dict(zip(Bus_info_data['정류장'], Bus_info_data['막차시간'])) 
last_bus_times = {stop: time_to_minutes(time_str) for stop, time_str in last_bus_times.items()} #시간 단위를 분으로 바꿔주기

## 모델링

### 모델을 돌리기 위한 시간, 가중치 범위 설정

In [160]:
# 시간 범위 (in minutes)
current_time_range = range(-30, 30, 5)  # From 23:30 pm to 0:30 am
# 가중치 범위
alpha_range = range(0, 300, 100) 
# big M 설정
M = 100000

### 모델 돌리기

In [ ]:
#parameter 단순화
S = bus_stops
H = buses
SEB = stops_each_bus
TC = taxi_costs
TT = taxi_times
BT = bus_times
WT = walk_times
LT = last_bus_times


for alpha in alpha_range:  #(0, 100, 200)
    results = []
    print(f'시간 가중치: {alpha}')
    for t in current_time_range:  #From 23:30 pm to 0:30 am

        # 최적화 모델 생성
        model = pulp.LpProblem("Minimize_Total_Cost_and_Time", sense=LpMinimize)
        
        # 결정 변수 설정
        X = pulp.LpVariable.dicts("X", (H, S, S), cat='Binary')  # h 버스를 타고 i정류장에서 탑승해 j정류장에서 하차(Binary variable )
        Z = pulp.LpVariable.dicts("Z", H, cat='Binary')  # h 버스를 탐(Binary variable )

        # 목적식 설정
        model += pulp.lpSum((TC[i] + 2500 + alpha * (TT[i] + BT[(i, j)] + WT[j])) * X[h][i][j] for h in H for i in S for j in S)

        # 제약식 설정
        
        # 모든 경로 중 하나 선택
        model += pulp.lpSum(X[h][i][j] for h in H for i in S for j in S) == 1
        # 모든 버스 중 하나 선택
        model += pulp.lpSum(Z[h] for h in H) == 1
        # 모든 버스에 대해
        for h in H:
            # h 버스를 탈 경우 h 버스의 노선끼리의 조합 중 하나 선택
            model += pulp.lpSum(X[h][i][j] for j in Nh[h] for i in Nh[h]) == Z[h]
            # 모든 탑승 정류장과 모든 하차 정류장에 대해
            for i in S:
                for j in S:
                    #거꾸로 가는 것을 방지
                    model += BT[(i, j)] * X[h][i][j] >= 0
                    # 탑승 정류장까지의 택시비는 30000원 이하
                    model += X[h][i][j] * TC[i] <= 30000
                    # 하차 정류장부터 걷는 시간이 20분 이하
                    model += X[h][i][j] * WT[j] <= 20
                    # 막차시간 내로 택시가 i 정류장에 도착해야 함
                    model += TT[i] <= LT[i] - t + M * (1 - X[h][i][j])
        # 문제 풀기
        model.solve()
 
        
        # 결과 나타내기
        
        # 시간 형태 바꾸기
        if t < 0:
            t = f'23:{t % 60:02d}'
        elif t >= 0:
            t = f'{t // 60}:{t % 60:02d}'
        
        # feasible 할 경우
        if LpStatus[model.status] == 'Optimal':
            for h in H:
                for i in S:
                    for j in S:
                        # 최적화 솔루션이 h버스 i정류장 탑승 j정류장 하차 경로일 경우
                        if X[h][i][j].value() == 1:
                            results.append({'현재시간': t ,'탑승 버스': h, '탑승 정류장': i, '하차 정류장': j, '택시비(원)': int(round(TC[i])), '총 이동시간(분)': TT[i] + BT[(i, j)] + WT[j]})
        # infeasible 할 경우
        else:
            results.append({'현재시간': t ,'탑승 버스': 'null', '탑승 정류장': 'null', '하차 정류장': 'null', '택시비(원)': int(57600), '총 이동시간(분)': 50})
        

    # 결과 DataFrame 생성
    results_df = pd.DataFrame(results)

    # pandas 출력 옵션 설정 (컬럼 너비 자동 조절)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.expand_frame_repr', False)
    pd.set_option('display.max_colwidth', None)
    
    # 출력 너비 조정
    pd.set_option('display.max_colwidth', 5)
    
    # 결과 출력
    print(results_df.to_string(index=False))




시간 가중치: 0
 현재시간 탑승 버스 탑승 정류장            하차 정류장  택시비(원)  총 이동시간(분)
23:30     2 남대문세무서   서원마을현대홈타운.삼성쉐르빌   10868         82
23:35     2 남대문세무서   서원마을현대홈타운.삼성쉐르빌   10868         82
23:40     2 남대문세무서   서원마을현대홈타운.삼성쉐르빌   10868         82
23:45     2 남대문세무서 현대성우5차.만현10단지아이파크   10868         81
23:50     2 남대문세무서 현대성우5차.만현10단지아이파크   10868         81
23:55     2 남대문세무서 현대성우5차.만현10단지아이파크   10868         81
 0:00     2 남대문세무서 현대성우5차.만현10단지아이파크   10868         81
 0:05  null   null              null   57600         50
 0:10  null   null              null   57600         50
 0:15  null   null              null   57600         50
 0:20  null   null              null   57600         50
 0:25  null   null              null   57600         50
시간 가중치: 100
